### Supply Chain Optimization

In [1]:
import pandas as pd
from pulp import *
import geopy.distance
from geopy.geocoders import Nominatim
import folium

##### Part 1: Making up some data for an example

In [2]:
#return data frame with coords given a list of cities
def get_coords(city_list):
    df = pd.DataFrame({'city': [],'lat': [],'long': []})
    geolocator = Nominatim(user_agent="supply_app")
    for city in city_list:
        location = geolocator.geocode(city)
        df2 = {'city': city, 'lat': location.latitude, 'long': location.longitude}
        df = df.append(df2, ignore_index = True)
    return df

In [3]:
#random suppliers, distribution centers, and customers with demand

suppliers = get_coords(['Des Moines', 'Iowa City', 'Cleveland', 'San Antonio'])
distributions = get_coords(['Houston'])
consumers = get_coords(['Destin', 'LA', 'Boston', 'Minneapolis', 'San Francisco', 'Miami'])

# give consumers some number of demand
consumers['demand'] = [100, 150, 50, 75, 100, 50]

all_cities = pd.concat([suppliers, distributions, consumers])

In [4]:
# refactor-- ideally want all of this in a dataframe with the decision vars stored there so I can reference city data 
# rather than writing new functions to go get it like the one below

##### Part 2: Helper Functions

In [5]:
# return distance in miles between two cities from our initial dataframe
# kind of messy way to do this
def get_distance(city1, city2):
    coords_1 = (all_cities[all_cities['city'] == city1].lat.iloc[0], all_cities[all_cities['city'] == city1].long.iloc[0])
    coords_2 = (all_cities[all_cities['city'] == city2].lat.iloc[0], all_cities[all_cities['city'] == city2].long.iloc[0])
    return geopy.distance.distance(coords_1, coords_2).miles

##### Part 3: Build the Model

In [6]:
# Initialize model
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("supplier_", [(i,j) for i in suppliers['city'] for j in distributions['city']],
                     lowBound=0, upBound=None)

y = LpVariable.dicts("distribution_", 
                     [(i,j) for i in distributions['city'] for j in consumers['city']], 
                    lowBound=0, upBound=None)

In [7]:
# Define Objective Function
COST_PER_MILE = 17 #Arbitrary mileage cost
model += (lpSum([get_distance(i,j) * x[(i,j)] * COST_PER_MILE for i in suppliers['city'] for j in distributions['city']]) + \
          lpSum([get_distance(i,j) * y[(i,j)] * COST_PER_MILE for i in distributions['city'] for j in consumers['city']]))

In [8]:
# Add Constraints

# Must meet demand at every consumer location
for consumer_city in consumers['city']:
    model += lpSum([y[(i, consumer_city)] for i in distributions['city']]) >= \
             consumers[consumers['city']==consumer_city].demand.iloc[0]

# Must recieve enough supply to distribute
for distribution_center in distributions['city']:
    model += lpSum([x[(i, distribution_center)] for i in suppliers['city']]) >= \
             lpSum([y[(distribution_center, j)] for j in consumers['city']])
    
# Do suppliers or distributers have limits?

##### Part 4: Solve the Model

In [9]:
# Solve Model
model.solve()
print("Total Cost = ${:,}".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

INITIAL_TOTAL_COST = int(value(model.objective))

Total Cost = $12,432,260

Status: Optimal


In [10]:
# gross way to do this, but works for now
def get_routes(model):
    routes = []
    for v in model.variables():
        if v.varValue > 0:
            cities = v.name.replace("_", " ").split("'")[1::2]
            routes = routes + [cities]
    return routes

##### Part 5: Visualize the Result

In [11]:
# use the Map function from folium to generate a map
# create map object with folium.Map()
mapObject = folium.Map(location = [29,-95],
                      zoom_start = 4)
# create markers with .Marker
for city in suppliers.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="blue",icon="cloud")
                             ).add_to(mapObject)
for city in distributions.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="red",icon="cloud")
                             ).add_to(mapObject)

for city in consumers.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="green",icon="cloud")
                             ).add_to(mapObject)
# add marker to map
def build_line(city1, city2):
    coords_1 = [all_cities[all_cities['city'] == city1].lat.iloc[0], all_cities[all_cities['city'] == city1].long.iloc[0]]
    coords_2 = [all_cities[all_cities['city'] == city2].lat.iloc[0], all_cities[all_cities['city'] == city2].long.iloc[0]]
    folium.PolyLine(locations=[coords_1, coords_2],weight=3).add_to(mapObject)
    return


# Create the map and add the line
routes = get_routes(model)

for route in routes:
    build_line(route[0], route[1])
    

In [23]:
# display map with marker
mapObject.save("index.html")

In [27]:
# from IPython.display import HTML
# display(HTML(filename='index.html'))
# import IPython
IPython.display.HTML(filename='index.html')

##### Now do it again with new customers or distributions

In [13]:
new_distributions = ['St Louis']
df = get_coords(new_distributions)
distributions = distributions.append(df, ignore_index=True)

# new_consumers = ['San Francisco', 'Miami']
# new_consumers_demand = [100, 50]
# df = get_coords(new_consumers)
# df['demand'] = new_consumers_demand
# consumers = consumers.append(df, ignore_index=True)

all_cities = pd.concat([suppliers, distributions, consumers])

In [14]:
# Initialize model
model = LpProblem("Supply_Chain_Optimization", LpMinimize)
# Create Decision Variables
x = LpVariable.dicts("supplier_", [(i,j) for i in suppliers['city'] for j in distributions['city']],
                     lowBound=0, upBound=None)

y = LpVariable.dicts("distribution_", 
                     [(i,j) for i in distributions['city'] for j in consumers['city']], 
                    lowBound=0, upBound=None)
# Define Objective Function
COST_PER_MILE = 17 #Arbitrary mileage cost
model += (lpSum([get_distance(i,j) * x[(i,j)] * COST_PER_MILE for i in suppliers['city'] for j in distributions['city']]) + \
          lpSum([get_distance(i,j) * y[(i,j)] * COST_PER_MILE for i in distributions['city'] for j in consumers['city']]))
# Add Constraints

# Must meet demand at every consumer location
for consumer_city in consumers['city']:
    model += lpSum([y[(i, consumer_city)] for i in distributions['city']]) >= \
             consumers[consumers['city']==consumer_city].demand.iloc[0]

# Must recieve enough supply to distribute
for distribution_center in distributions['city']:
    model += lpSum([x[(i, distribution_center)] for i in suppliers['city']]) >= \
             lpSum([y[(distribution_center, j)] for j in consumers['city']])
    
# Do suppliers or distributers have limits?

# Solve Model
model.solve()
print("Total Cost = ${:,}".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

NEW_TOTAL_COST = int(value(model.objective))

Total Cost = $11,257,338

Status: Optimal


In [15]:
# use the Map function from folium to generate a map
# create map object with folium.Map()
mapObject = folium.Map(location = [29,-95],
                      zoom_start = 4)
# create markers with .Marker
for city in suppliers.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="blue",icon="cloud")
                             ).add_to(mapObject)
for city in distributions.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="red",icon="cloud")
                             ).add_to(mapObject)

for city in consumers.city:
    folium.Marker(location= [all_cities[all_cities['city'] == city].lat.iloc[0], all_cities[all_cities['city'] == city].long.iloc[0]],
                             popup = "This is a marker!",
                             icon=folium.Icon(color="green",icon="cloud")
                             ).add_to(mapObject)
# add marker to map
def build_line(city1, city2):
    coords_1 = [all_cities[all_cities['city'] == city1].lat.iloc[0], all_cities[all_cities['city'] == city1].long.iloc[0]]
    coords_2 = [all_cities[all_cities['city'] == city2].lat.iloc[0], all_cities[all_cities['city'] == city2].long.iloc[0]]
    folium.PolyLine(locations=[coords_1, coords_2],weight=3).add_to(mapObject)
    return


# Create the map and add the line
routes = get_routes(model)

for route in routes:
    build_line(route[0], route[1])
    

In [16]:
# display map with marker
mapObject

In [17]:
print("Cost difference is: ${:,}".format(INITIAL_TOTAL_COST - NEW_TOTAL_COST))

Cost difference is: $1,174,922


In [209]:
class Folium_Mapper:
    def __init__(self, suppliers, distributions, consumers, all_cities, model):
        self.suppliers = suppliers
        self.distributions = distributions
        self.consumers = consumers
        self.all_cities = all_cities
        self.model = model
        self.mapObject = None

# use the Map function from folium to generate a map
# create map object with folium.Map()
    def build_map(self):
        mapObject = folium.Map(location = [29,-95],
                              zoom_start = 4)
        # create markers with .Marker
        for city in self.suppliers.city:
            folium.Marker(location= [self.all_cities[self.all_cities['city'] == city].lat.iloc[0], self.all_cities[self.all_cities['city'] == city].long.iloc[0]],
                                     popup = "This is a marker!",
                                     icon=folium.Icon(color="blue",icon="cloud")
                                     ).add_to(mapObject)
        for city in self.distributions.city:
            folium.Marker(location= [self.all_cities[self.all_cities['city'] == city].lat.iloc[0], self.all_cities[self.all_cities['city'] == city].long.iloc[0]],
                                     popup = "This is a marker!",
                                     icon=folium.Icon(color="red",icon="cloud")
                                     ).add_to(mapObject)
        
        for city in self.consumers.city:
            folium.Marker(location= [self.all_cities[self.all_cities['city'] == city].lat.iloc[0], self.all_cities[self.all_cities['city'] == city].long.iloc[0]],
                                     popup = "This is a marker!",
                                     icon=folium.Icon(color="green",icon="cloud")
                                     ).add_to(mapObject)
        # add marker to map
        def build_line(city1, city2):
            coords_1 = [self.all_cities[self.all_cities['city'] == city1].lat.iloc[0], self.all_cities[self.all_cities['city'] == city1].long.iloc[0]]
            coords_2 = [self.all_cities[self.all_cities['city'] == city2].lat.iloc[0], self.all_cities[self.all_cities['city'] == city2].long.iloc[0]]
            folium.PolyLine(locations=[coords_1, coords_2],weight=3).add_to(mapObject)
            return
        
        # gross way to do this, but works for now
        def get_routes(model):
            routes = []
            for v in model.variables():
                if v.varValue > 0:
                    cities = v.name.replace("_", " ").split("'")[1::2]
                    routes = routes + [cities]
            return routes
        
        # Create the map and add the line
        routes = get_routes(self.model)
        
        for route in routes:
            build_line(route[0], route[1])
            
        self.mapObject = mapObject
        
    def save_map(self):
        self.mapObject.save("index.html")


In [210]:
mapper = Folium_Mapper(suppliers, distributions, consumers, all_cities, o.model)


In [211]:
mapper.build_map()


In [212]:
mapper.save_map()

In [215]:
o.solved_model.objective

AttributeError: 'Optimizer' object has no attribute 'solved_model'